In [4]:
import dicom2nifti
import os
from pathlib import Path
import os
import numpy as np
import matplotlib.pyplot as plt
import imageio
import glob
import nibabel as nib
from itertools import chain
import SimpleITK as sitk
import imagej
import nrrd
from sklearn.mixture import GaussianMixture
from ipywidgets import interact, fixed
from IPython.display import display

import SimpleITK as sitk
import imagej
# from PIL import Image
import pydicom as dicom
import os
import numpy
from matplotlib import pyplot, cm
from rt_utils import RTStructBuilder

In [5]:
import dicom2nifti
import os
from pathlib import Path
import os
import numpy as np
import matplotlib.pyplot as plt
import imageio
import glob
import nibabel as nib
from itertools import chain
from util.imgsitk import *
import SimpleITK as sitk
import imagej
from ipywidgets import interact, fixed
import itk
from IPython.display import clear_output
from util.spineSeg import *
from util.spineSegC import *
from lungmask import mask
from rt_utils import RTStructBuilder
import matplotlib.pyplot as plt
from utl.pytorch_morph import *

In [6]:
    num_of_projections = 360
    angles = np.linspace(0, 2 * np.pi, num=num_of_projections, endpoint=False)
    angles = angles[[330,0,30]]
    angles

array([5.75958653, 0.        , 0.52359878])

In [7]:
def display_images_with_alpha_numpy(image_z, alpha, fixed, moving):
    img = (1.0 - alpha)*fixed[image_z,:,:] + alpha*moving[image_z,:,:] 
    plt.figure(figsize=(16,9))
    plt.imshow(img,cmap=plt.cm.Greys_r);
    # x = plt.ginput(3)
    # print(x)
    # plt.ginput()
    # plt.axis('off')
    plt.show()

def win_scale(data, wl, ww):
    """
    Scale pixel intensity data using specified window level, width, and intensity range.
    """
    datatype=data.dtype
    image=sitk.GetImageFromArray(data)

    resacleFilter = sitk.IntensityWindowingImageFilter()
    resacleFilter.SetWindowMaximum(wl+ww/2)
    resacleFilter.SetWindowMinimum(wl-ww/2)
    resacleFilter.SetOutputMaximum(1)
    resacleFilter.SetOutputMinimum(0)

    out = resacleFilter.Execute(image)
    
    data_new=sitk.GetArrayFromImage(out)

    return data_new.astype(datatype)

def scale(x, out_range=(-1, 1)):
    domain = np.min(x), np.max(x)
    y = (x - (domain[1] + domain[0]) / 2) / (domain[1] - domain[0])
    return y * (out_range[1] - out_range[0]) + (out_range[1] + out_range[0]) / 2


class Imshow3D:
    def __init__(self,fig, ax, vol):

        self.fig=fig
        self.connect()
        self.ax = ax
        ax.set_title('use scroll wheel to navigate images')

        self.vol = vol
        self.vol_bk = vol

        self.maxvol = vol.max()
        self.minvol = vol.min()

        rows, cols, self.slices = vol.shape
        self.ind = self.slices//2


        self.maxlevel=450
        self.minlevel=-350
        self.im = ax.imshow(self.vol[:, :, self.ind], cmap='gray')
        

        self.sliders=widgets.IntSlider(description='Slice Number',value=self.ind ,min=0, max=self.slices, step=1,)
        self.sliderl=widgets.IntSlider(description='Level',value=50 ,min=self.minvol, max=self.maxvol, step=1,)
        self.sliderw=widgets.IntSlider(description='Window',value=400 ,min=1, max=self.maxvol//2, step=1,)
        # self.sliders =self.ind
        # l = widgets
        # ((sliders1, 'value')
        self.sliders.observe(self.handle_slider_change, names='value')

        self.sliderl.observe(self.handle_level_window, names='value')
        self.sliderw.observe(self.handle_level_window, names='value')

        display(self.sliders,self.sliderl,self.sliderw)
        self.update()

    def connect(self):
        
        """Connect to all the events we need."""
        self.cidpress = self.fig.canvas.mpl_connect('button_press_event', self.on_press)

        self.cidrelease = self.fig.canvas.mpl_connect('button_release_event', self.on_release)

        self.cidmotion = self.fig.canvas.mpl_connect('motion_notify_event', self.on_motion)

        self.onscroll=self.fig.canvas.mpl_connect('motion_notify_event', self.on_scroll)

        self.onkey=self.fig.canvas.mpl_connect('key_press_event', self.on_key)

    def on_scroll(self, event):
        # print("%s %s" % (event.button, event.step))
        # if event.button == 'up' :
        #     self.ind = (self.ind + 1) % self.slices
        # elif event.button == 'down' :
        #     self.ind = (self.ind - 1) % self.slices
        
        # self.update()
        True


        # if event.motion == 'up':
        #     self.ind = (self.ind + 1) % self.slices

    def on_press(self, event):
        True

    def on_release(self, event):
        True

    def on_motion(self, event):
        True

    def on_key (self, event):

        if event.key == 'up':
            self.ind = (self.ind + 1) % self.slices
        elif event.key == 'down':
            self.ind = (self.ind - 1) % self.slices
        # self.fig.canvas.draw()
        self.update()

    # def previous_slice(self,ax):
    #     """Go to the previous slice."""
    #     volume = ax.volume
    #     ax.index = (ax.index - 1) % volume.shape[0]  # wrap around using %
    #     self.ax.images[0].set_array(volume[ax.index])

    # def next_slice(self,ax):
    #     """Go to the next slice."""
    #     volume = ax.volume
    #     ax.index = (ax.index + 1) % volume.shape[0]
    #     self.ax.images[0].set_array(volume[ax.index])
    
    def handle_slider_change(self, change):
        True
        # print(self.sliders.value)
        self.ind=self.sliders.value
        self.update()


    def handle_level_window(self, change):
        """
        Function to display an image slice
        Input is a numpy 2D array
        """
        self.vol=self.vol_bk

        level = self.sliderl.value
        window = self.sliderw.value
        self.maxlevel = level + window/2
        self.minlevel = level - window/2

        if self.maxvol<self.maxlevel:
            self.maxlevel=self.maxvol
        if self.minvol>self.minlevel:
            self.minlevel=self.minvol

            



        self.update()


    def update(self):
        self.sliders.value=self.ind
        self.im.set_data(self.vol[:, :, self.ind])
        self.im.set_clim(vmin=self.minlevel,vmax=self.maxlevel)
        self.ax.set_ylabel('slice %s' % self.ind)
        plt.autoscale()
        self.im.axes.figure.canvas.draw()
        # print(self.sliders.value)




def rescale(img_data1q,stc):
    sz=img_data1q.shape
    img_data1qt=torch.from_numpy(img_data1q)
    astc=np.abs(np.round(stc)).astype(int)
    istc=tuple(np.round(-stc).astype(int))

    img_data1qt=torch.nn.functional.pad(img_data1qt.unsqueeze(0).unsqueeze(0),(0,astc[2], 0,astc[1], 0,astc[0]),mode='replicate')
    img_data1qt=torch.roll(img_data1qt, shifts=istc, dims=(2,3,4) )
    img_data1qt=img_data1qt[:,:,0:sz[0],0:sz[1],0:sz[2]].squeeze(0).squeeze(0).numpy()
    return img_data1qt


Creating the 3 projections for input to CNN with no enhancement or spacer simulation


In [8]:

import astra
import torch
import torchvision.transforms as transforms
data_folder="./TestDataNii/"
list_CT=Path(data_folder).glob('**/*_CTD.nii.gz')

DataInfoTuple=[]
n=0
for i in list_CT:
    DataInfoTuple.append(i)
    n=n+1

# for i,j in DataInfoTuple:
for kkk in range(0,2):
    # kkk=14
    i=DataInfoTuple[kkk]

    s=str(i).find('\P')
    e=str(i).find('_IMG')
    casenum=int(str(i)[s+2:e])
    print(['Case Number is ' + str(casenum)])
    img_data = nib.load(i).get_fdata()

    img_data1=np.moveaxis(img_data,2,0)
    img_data1=np.moveaxis(img_data1,1,2)
    img_data1=np.flip(img_data1,1)
    stc=np.array((-30,0,0))
    img_data1=rescale(img_data1.copy(),stc)
    matct=img_data1.astype('float32')
    # matct[np.where(matct<-1000)]=-1000
    matct=np.where(matct < -1000, -1000, matct)

    # matct=matct-1000
    matct=(matct/1000)*0.2+0.2

    # mask_rs=mask_3d1
    matct_rs=matct

    sz=matct_rs.shape
    # sitk.Show(sitk.GetImageFromArray(matct))
    print(sz)

    SOD = 1000  # [mm]
    ODD = 500  # [mm]
    DS = 1  # [mm]
    ## then each pixel on detector corresponds to SOD/(SOD+ODD) mm of the object
    detector_rows = 512  # Vertical size of detector [pixels].
    detector_cols = 512  # Horizontal size of detector [pixels].
    num_of_projections = 360
    angles = np.linspace(0, 2 * np.pi, num=num_of_projections, endpoint=False)
    angles = angles[[330,0,30]]
    sz=matct_rs.shape


    proj_geom = \
    astra.create_proj_geom('cone', DS, DS, 512, 512, angles,SOD,ODD)
                            #  (distance_source_origin + distance_origin_detector) /
                            #  detector_pixel_size, 0)

    vol_geom = astra.creators.create_vol_geom(sz[1], sz[2],
                                            sz[0])

    ## Image Projection
    phantom_id = astra.data3d.create('-vol', vol_geom, data=matct_rs)

    projections_id, projections = \
    astra.creators.create_sino3d_gpu(phantom_id, proj_geom, vol_geom)

    projections = (projections-projections.mean()) / projections.std()
    # projections=np.clip(projections,-3,3)/3
    projections = (projections-projections.min())/(projections.max()-projections.min())

    projc=np.moveaxis(projections,1,0)

    for i in range(projc.shape[0]):
        projc[i,:,:]=np.flipud(projc[i,:,:])

    Savedir='TestData/'

    for i in range(projc.shape[0]):
        nimg = nib.Nifti1Image(projc[i,:,:], np.eye(4))
        filename='P' + str(casenum).zfill(2)+ '_' + 'CTD' + '_' + str(i) 
        nib.save(nimg, Savedir + filename)



pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1


['Case Number is 0']
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
(432, 454, 454)


pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1


['Case Number is 1']
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
(421, 382, 382)


The projection of Diagnostic CT with spacer enhancement for planned location of spacer

In [9]:

import astra
import torch
import torchvision.transforms as transforms
data_folder="./TestDataNii/"
list_CT=Path(data_folder).glob('**/*_CTDh.nii.gz')

DataInfoTuple=[]
n=0
for i in list_CT:
    DataInfoTuple.append(i)
    n=n+1

# for i,j in DataInfoTuple:
for kkk in range(0,2):
    # kkk=14
    i=DataInfoTuple[kkk]

    s=str(i).find('\P')
    e=str(i).find('_IMG')
    casenum=int(str(i)[s+2:e])
    print(['Case Number is ' + str(casenum)])
    img_data = nib.load(i).get_fdata()

    img_data1=np.moveaxis(img_data,2,0)
    img_data1=np.moveaxis(img_data1,1,2)
    img_data1=np.flip(img_data1,1)
    stc=np.array((-30,0,0))
    img_data1=rescale(img_data1.copy(),stc)
    matct=img_data1.astype('float32')
    # matct[np.where(matct<-1000)]=-1000
    matct=np.where(matct < -1000, -1000, matct)

    # matct=matct-1000
    matct=(matct/1000)*0.2+0.2

    # mask_rs=mask_3d1
    matct_rs=matct

    sz=matct_rs.shape
    # sitk.Show(sitk.GetImageFromArray(matct))
    print(sz)

    SOD = 1000  # [mm]
    ODD = 500  # [mm]
    DS = 1  # [mm]
    ## then each pixel on detector corresponds to SOD/(SOD+ODD) mm of the object
    detector_rows = 512  # Vertical size of detector [pixels].
    detector_cols = 512  # Horizontal size of detector [pixels].
    num_of_projections = 360
    angles = np.linspace(0, 2 * np.pi, num=num_of_projections, endpoint=False)
    angles = angles[[330,0,30]]
    sz=matct_rs.shape


    proj_geom = \
    astra.create_proj_geom('cone', DS, DS, 512, 512, angles,SOD,ODD)
                            #  (distance_source_origin + distance_origin_detector) /
                            #  detector_pixel_size, 0)

    vol_geom = astra.creators.create_vol_geom(sz[1], sz[2],
                                            sz[0])

    ## Image Projection
    phantom_id = astra.data3d.create('-vol', vol_geom, data=matct_rs)

    projections_id, projections = \
    astra.creators.create_sino3d_gpu(phantom_id, proj_geom, vol_geom)

    projections = (projections-projections.mean()) / projections.std()
    # projections=np.clip(projections,-3,3)/3
    projections = (projections-projections.min())/(projections.max()-projections.min())

    projc=np.moveaxis(projections,1,0)

    for i in range(projc.shape[0]):
        projc[i,:,:]=np.flipud(projc[i,:,:])

    Savedir='TestData/'

    for i in range(projc.shape[0]):
        nimg = nib.Nifti1Image(projc[i,:,:], np.eye(4))
        filename='P' + str(casenum).zfill(2)+ '_' + 'CTDh' + '_' + str(i) 
        nib.save(nimg, Savedir + filename)



pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1


['Case Number is 0']
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1


pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1


(432, 454, 454)
['Case Number is 1']
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
(421, 382, 382)
